#**Aula 3 - Construindo o modelo**

Importando as bibliotecas necessárias:


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

Importando os dados:

In [ ]:
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo
doenca_cardiaca = fetch_ucirepo(id=45)

Criando a variável `X`, que contém as features e o target:

In [ ]:
X = doenca_cardiaca.data.features
X = X[['age', 'chol', 'cp']]

target = doenca_cardiaca.data.targets
target = (target > 0) * 1

Transformando a coluna `cp` do DataFrame `X` em variáveis dummy:

In [ ]:
df_cp = pd.get_dummies(X['cp'], prefix='cp')
X = X.drop('cp', axis=1).join(df_cp)

Criando uma constante `y` com o target:

In [ ]:
y = tf.constant(target, dtype=tf.float32)

##**Normalizando os dados**

Chegou a hora de incluir todas as variáveis como entrada!

Visualizando as 5 primeiras linhas de `X`:

In [ ]:
X.head()

age  chol  cp_1  cp_2  cp_3  cp_4
0   63   233     1     0     0     0
1   67   286     0     0     0     1
2   67   229     0     0     0     1
3   37   250     0     0     1     0
4   41   204     0     1     0     0

Normalizando os valores das colunas `age` e `chol` com StandardScaler():

> A classe [StandardScaler()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) realiza essa padronização através da subtração da média e divisão pelo desvio padrão de cada atributo. Isso faz com que os dados tenham uma média aproximadamente zero e um desvio padrão aproximadamente igual a um.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [ ]:
X[['age', 'chol']] = scaler.fit_transform(X[['age', 'chol']])

print('Primeiros valores normalizados de age:', X['age'].head())
print('Primeiros valores normalizados de chol:', X['chol'].head())

Primeiros valores normalizados de age: 0    0.948726
1    1.392002
2    1.392002
3   -1.932564
4   -1.489288
Name: age, dtype: float64
Primeiros valores normalizados de chol: 0   -0.264900
1    0.760415
2   -0.342283
3    0.063974
4   -0.825922
Name: chol, dtype: float64


##**Obtendo a perda**

Checando quantas variáveis de entrada nós temos em `X`:

In [ ]:
X.shape[1]

6

Definindo uma semente para a geração de números aleatórios, garantindo a reprodutibilidade dos resultados. Em seguida, criando variáveis para os pesos e vies do neurônio.


A função `neuronio()` calcula a saída do neurônio para uma entrada `x`, aplicando uma transformação linear (produto matricial entre `x` e `peso`, seguido da adição do `vies`):

In [ ]:
tf.random.set_seed(31)

quantidade_features = X.shape[1]
peso = tf.Variable(tf.random.normal([quantidade_features, 1]), name='peso')
vies = tf.Variable(tf.random.normal([1]), name='vies')

def neuronio(x):
  z = tf.add(tf.matmul(x, peso), vies)
  return z

Visualizando o viés e o pesos obtidos:

In [ ]:
vies.numpy(), peso.numpy()

(array([0.15100643], dtype=float32),
 array([[-0.3594139 ],
        [ 0.18532915],
        [ 2.3871737 ],
        [ 0.9996345 ],
        [ 0.09981808],
        [-0.06186233]], dtype=float32))

Vamos testar classificar o primeiro paciente em `X`?

Verificando os valores de cada variável do paciente 0, temos os seguintes valores:

In [ ]:
X.iloc[0].values

array([ 0.94872647, -0.2649003 ,  1.        ,  0.        ,  0.        ,
        0.        ])

Passando valores à função `neuronio`:

In [ ]:
neuronio(tf.constant([[0.94872647, -0.2649003,  1.0,  0.0,  0.0, 0.0]]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.1481009]], dtype=float32)>

Obtendo a probabilidade do primeiro paciente ter a doença cardíaca:

In [ ]:
probabilidade = tf.sigmoid(neuronio(tf.constant([[0.94872647, -0.2649003,  1.0,  0.0,  0.0, 0.0]])))
probabilidade

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.8954912]], dtype=float32)>

Agora, utilizando a entrada que contempla todos os pacientes e exibindo as probabilidades dos dez primeiros:

In [ ]:
entrada = tf.constant(X, dtype=tf.float32)
probabilidade = tf.sigmoid(neuronio(entrada))
probabilidade[:10].numpy()

array([[0.8954912 ],
       [0.43284974],
       [0.38352907],
       [0.7225751 ],
       [0.82242215],
       [0.7407992 ],
       [0.46614692],
       [0.5919096 ],
       [0.44382718],
       [0.4974511 ]], dtype=float32)

Calculando a perda:

In [ ]:
calculadora_perda = tf.keras.losses.BinaryCrossentropy()
perda = calculadora_perda(y_true=y, y_pred=probabilidade)
perda.numpy()

0.9295621

Obtendo a acurácia:

In [ ]:
np.mean(y.numpy() == ((probabilidade.numpy() > 0.5)*1))

0.3696369636963696

##**Aplicando o gradiente descendente**

Inicializando o ambiente para um modelo de rede neural, definindo uma semente aleatória para garantir a reprodutibilidade dos resultados. Em seguida, definindo a estrutura de um neurônio simples, incluindo a inicialização dos pesos e do viés com valores aleatórios normais, e a função `neuronio`, que calcula a saída `z` a partir de entradas `x`.

Por fim, criando uma função para calcular a perda com a entropia cruzada binária:

In [ ]:
tf.random.set_seed(31)

quantidade_features = X.shape[1]

pesos = tf.Variable(tf.random.normal([quantidade_features,1]), name='pesos')
vies = tf.Variable(tf.random.normal([1]), name='vies')

def neuronio(x):
  z = tf.add(tf.matmul(x, pesos), vies)
  return z

calculadora_perda = tf.keras.losses.BinaryCrossentropy()

Definindo os dados de entrada:

In [ ]:
entrada = tf.constant(X, dtype=tf.float32)

Visualizando os três primeiros registros nos tensores `entrada` e `y`:

In [ ]:
entrada[:3], y[:3]

(<tf.Tensor: shape=(3, 6), dtype=float32, numpy=
 array([[ 0.9487265, -0.2649003,  1.       ,  0.       ,  0.       ,
          0.       ],
        [ 1.3920019,  0.7604152,  0.       ,  0.       ,  0.       ,
          1.       ],
        [ 1.3920019, -0.3422826,  0.       ,  0.       ,  0.       ,
          1.       ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 1), dtype=float32, numpy=
 array([[0.],
        [1.],
        [1.]], dtype=float32)>)

Definindo a taxa de aprendizado como 0.01. Em seguida, criando um otimizador utilizando o método de Descida do Gradiente Estocástico (SGD) do TensorFlow, passando a taxa de aprendizado como parâmetro:
> O otimizador será usado para atualizar os parâmetros do modelo durante o treinamento, visando minimizar a função de perda.

In [ ]:
learning_rate = 0.01

otimizador = tf.optimizers.SGD(learning_rate=learning_rate)

Definindo uma variável chamada `QUANTIDADE_EPOCAS` com o valor de 1000:
> Essa constante será utilizada para determinar o número total de épocas (ou iterações) no processo de treinamento de modelo, indicando quantas vezes o conjunto de dados será passado pelo modelo para ajustar os pesos das conexões na tentativa de minimizar o erro de predição.

In [ ]:
QUANTIDADE_EPOCAS = 1000

Realizando o treinamento do modelo de rede neural através de um loop que itera por uma quantidade definida de épocas:
> A cada época, ele utiliza o `tf.GradientTape` para monitorar as operações para calcular os gradientes do custo em relação aos pesos e vieses do neurônio, aplicando posteriormente esses gradientes para otimizar os parâmetros do modelo com o objetivo de minimizar a função de perda. A cada 100 épocas, o código imprime o número da época e o valor atual do custo, permitindo acompanhar a evolução do treinamento.

In [ ]:
for epoca in range(QUANTIDADE_EPOCAS):
  with tf.GradientTape() as tape:
    probabilidade = tf.sigmoid(neuronio(entrada))
    custo = calculadora_perda(y, probabilidade)


  gradientes = tape.gradient(custo, [pesos, vies])
  otimizador.apply_gradients(zip(gradientes, [pesos, vies]))

  if (epoca + 1) % 100 == 0:
    print(f'Época: {epoca+1}, Custo: {custo.numpy()}')

Época: 100, Custo: 0.8540225625038147
Época: 200, Custo: 0.7985237836837769
Época: 300, Custo: 0.7575094699859619
Época: 400, Custo: 0.7262797951698303
Época: 500, Custo: 0.7017260193824768
Época: 600, Custo: 0.6818602681159973
Época: 700, Custo: 0.6654065847396851
Época: 800, Custo: 0.6515259742736816
Época: 900, Custo: 0.6396480798721313
Época: 1000, Custo: 0.6293708086013794


Obtendo a acurácia:

In [ ]:
np.mean(y.numpy() == ((probabilidade.numpy() > 0.5)*1))

0.7161716171617162